In [ ]:
# Import libraries and read in datasets
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')



In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/amazon_reviews_multilingual_US_v1_00.tsv"

apr = pd.read_csv(path,
                       sep='\t', error_bad_lines=False, warn_bad_lines=False)

In [ ]:
# Find out how many downvotes each review received
apr['downvotes'] = apr.total_votes - apr.helpful_votes

# Reorder columns
columns = ['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'downvotes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date']

apr = apr[columns]

In [ ]:
# Standardized product category names
apr.product_category.replace({
        'Digital_Music_Purchase': 'Digital Music Purchase',
        'Digital_Ebook_Purchase': 'Digital eBook Purchase',
        'Digital_Video_Download': 'Digital Video Download',
        'Lawn and Garden': 'Lawn & Garden',
        'Mobile_Electronics': 'Mobile Electronics',
        'Personal_Care_Appliances': 'Personal Care Appliances',
        'Mobile_Apps': 'Mobile Apps'
    }, inplace=True)

apr.product_category.unique()

array(['Books', 'Music', 'Video', 'Video DVD', 'Toys', 'Tools',
       'Office Products', 'Video Games', 'Software',
       'Digital Music Purchase', 'Home Entertainment', 'Electronics',
       'Digital eBook Purchase', 'Digital Video Download', 'Kitchen',
       'Camera', 'Outdoors', 'Musical Instruments', 'Sports', 'Watches',
       'PC', 'Home', 'Wireless', 'Beauty', 'Baby', 'Home Improvement',
       'Apparel', 'Shoes', 'Lawn & Garden', 'Mobile Electronics',
       'Health & Personal Care', 'Grocery', 'Luggage',
       'Personal Care Appliances', 'Automotive', 'Mobile Apps',
       'Furniture', '2012-12-22', 'Pet Products'], dtype=object)

## A Look into Our Initial Dataset

In [ ]:
# An initial look at our dataframe
print('Reviews are taken from: {} to {}\n'. format(sorted(apr.review_date.head(1)).pop(),
                                                   sorted(apr.review_date.tail(1)).pop()))
print('Number of reviews: {:,}'.format(len(apr)))
print('Number of customers: {:,}'.format(len(apr.customer_id.unique())))
print('Number of product parents: {:,}'.format(len(apr.product_parent.unique())))
print('Number of product titles: {:,}'.format(len(apr.product_title.unique())))
print('Number of unique products: {:,}'.format(len(apr.product_id.unique())))
print('Number of product categories: {:,}'.format(len(apr.product_category.unique())))
print('Number of days a review was made on: {:,}'.format(len(apr.review_date.unique())))

Reviews are taken from: 1995-08-13 to 2015-08-31

Number of reviews: 6,900,886
Number of customers: 4,097,784
Number of product parents: 16,310
Number of product titles: 52,326
Number of unique products: 86,813
Number of product categories: 39
Number of days a review was made on: 6,916


In [ ]:
# Many lost unique product ids were actually small variations of the same product
apr.groupby('product_parent')['product_title'].value_counts()[12:20]

product_parent  product_title                                                        
896923          For Whom The Bell Tolls                                                  8
                FOR WHOM THE BELL TOLLS                                                  5
                For Whom the Bell Tolls (War Promo)                                      2
                FOR WHOM THE BELL TOLLS ~1940 BOOK CLUB EDITION                          1
                For Whom The Bell Tolls (Turtleback School & Library Binding Edition)    1
                For Whom the Bell Tolls (Illustrated)                                    1
                For Whom the Bell Tolls (Scribner Classic)                               1
                For Whom the Bell Tolls (The Scribner Library)                           1
Name: product_title, dtype: int64

The mean before dropping rows is 80 purchases per product, but we wanted a higher threshold; especially because a plethora of unique products were 1 purchase products.

In [ ]:
# Summary statistics of original dataset
apr.product_id.value_counts().describe()

count    86813.000000
mean        79.491390
std        570.110821
min          1.000000
25%          1.000000
50%          6.000000
75%         36.000000
max      49789.000000
Name: product_id, dtype: float64

In [ ]:
# Number of products purchased after dropping any unique products with less than 100 purchases
print('Number of purchases: {:,}'.format(
    apr.groupby('product_id')['product_id'].count()[apr.groupby('product_id')['product_id'].count() >= 100].sum()))
print('Number of unique products: {:,}'.format(
    len(apr.groupby('product_id')['product_id'].count()[apr.groupby('product_id')['product_id'].count() >= 100])))
print('Dataset reduced by {:.2f}%'.format((1-5833771/len(apr))*100))

Number of purchases: 5,834,243
Number of unique products: 11,502
Dataset reduced by 15.46%


In [ ]:
# Extract counts on how many customers purchased a unique product
purchased_counts = apr.product_id.value_counts().rename('purchased_counts')

# Add column containing product purchased counts
apr = apr.merge(purchased_counts.to_frame(),
                left_on='product_id',
                right_index=True)

# Filter only products with 100 or more purchases
apr = apr[apr.purchased_counts >= 100]

## A Look into Our New Dataset

In [ ]:
print('Reviews are taken from: {} to {}\n'. format(sorted(apr.review_date.head(1)).pop(),
                                                   sorted(apr.review_date.tail(1)).pop()))
print('Number of reviews: {:,}'.format(len(apr)))
print('Number of customers: {:,}'.format(len(apr.customer_id.unique())))
print('Number of product parents: {:,}'.format(len(apr.product_parent.unique())))
print('Number of product titles: {:,}'.format(len(apr.product_title.unique())))
print('Number of unique products: {:,}'.format(len(apr.product_id.unique())))
print('Number of product categories: {:,}'.format(len(apr.product_category.unique())))
print('Number of days a review was made on: {:,}'.format(len(apr.review_date.unique())))

Reviews are taken from: 1995-08-13 to 2015-08-31

Number of reviews: 5,834,243
Number of customers: 3,653,843
Number of product parents: 6,654
Number of product titles: 9,081
Number of unique products: 11,502
Number of product categories: 29
Number of days a review was made on: 6,839


In [ ]:
# New summary statistics
apr.product_id.value_counts().describe()

count    11502.000000
mean       507.237263
std       1496.494488
min        100.000000
25%        141.000000
50%        218.000000
75%        403.000000
max      49789.000000
Name: product_id, dtype: float64

In [ ]:
# Number of reviews in each product category
apr.groupby('product_category')['product_category'].count()

product_category
Automotive                    102
Baby                         5352
Books                      658781
Camera                      12997
Digital Music Purchase      39401
Digital Video Download     887119
Digital eBook Purchase    1170542
Electronics                  9303
Health & Personal Care        971
Home                         1384
Home Entertainment          35868
Home Improvement             3020
Kitchen                      1508
Lawn & Garden                1167
Mobile Apps               1451751
Music                      511926
Musical Instruments          9473
Office Products              1836
Outdoors                     2575
PC                          48917
Shoes                         281
Sports                       3483
Tools                        7023
Toys                        41353
Video                        9080
Video DVD                  874995
Video Games                 14693
Watches                      8756
Wireless                    205

In [ ]:
apr.groupby('product_parent')['product_id'].value_counts().head()

product_parent  product_id
225472          B00K7C17QW    4008
896923          0684803356     329
                B000FC0OOU     211
1002835         B000002GB9     132
1005285         B004PAHZMC     283
Name: product_id, dtype: int64

In [ ]:
apr.groupby('product_parent')['product_title'].value_counts().head()

product_parent  product_title                                                      
225472          CSI: Hidden Crimes                                                     4008
896923          For Whom the Bell Tolls                                                 329
                For Whom the Bell Tolls (Scribner Classics)                             211
1002835         The Grand Illusion                                                      132
1005285         Neat-Oh! LEGO Star Wars ZipBin Battle Bridge 500 Brick Storage Case     283
Name: product_title, dtype: int64

In [ ]:
apr.groupby('product_parent')['product_category'].value_counts().head()

product_parent  product_category      
225472          Mobile Apps               4008
896923          Books                      329
                Digital eBook Purchase     211
1002835         Music                      132
1005285         Toys                       283
Name: product_category, dtype: int64

In [ ]:
example = apr[apr.product_parent == 999720718]
example.groupby('product_parent')['product_title'].value_counts()

product_parent  product_title                              
999720718       Skipping Christmas: A Novel                    719
                The Grapes of Wrath                            706
                The Partner                                    494
                The Partner: A Novel                           453
                Skipping Christmas                             256
                The Grapes of Wrath (Centennial Edition)       205
                The Grapes of Wrath (20th Century Classics)    200
Name: product_title, dtype: int64

In [ ]:
example = apr[apr.product_parent == 382448153]
example.groupby('product_parent')['product_title'].value_counts()

product_parent  product_title                                                   
382448153       The Girl Who Kicked the Hornet's Nest (Millennium Series Book 3)    1804
                The Girl Who Played with Fire (Millennium Series Book 2)            1569
                The Girl Who Kicked the Hornet's Nest (Millennium Trilogy)          1093
                The Girl Who Played with Fire (Millennium Series)                   1091
                The Girl Who Played with Fire (Millennium )                          455
                The Girl Who Kicked the Hornet's Nest                                304
                The Girl Who Kicked the Hornets' Nest (Millennium Series Book 3)     199
                The Girl Who Kicked the Hornet's Nest (Millennium Series)            168
                The Girl Who Played With Fire (Millennium Series)                    152
Name: product_title, dtype: int64

In [ ]:
example = apr[apr.product_parent == 825691170]
example.groupby('product_parent')['product_title'].value_counts()

product_parent  product_title                                             
825691170       The Maze Runner (The Maze Runner, Book 1)                     5288
                The Kill Order (Prequel)                                       929
                The Maze Runner (Book 1)                                       905
                The Maze Runner Series Complete Collection (Maze Runner)       530
                The Maze Runner Series (Maze Runner)                           360
                The Maze Runner (Maze Runner, Book One)                        242
                The Kill Order: Book Four; Origin (The Maze Runner Series)     235
                The Maze Runner Trilogy (Maze Runner)                          193
                The Kill Order (The Maze Runner)                               136
Name: product_title, dtype: int64

In [ ]:
def missing(dataset):
    columns = dataset.columns
    print('MISSING ROWS per COLUMN')
    for column in columns:
        percentage = (dataset[column].isnull().sum() / len(dataset)) * 100
        print('{}: {}, {:0.2f}%'.format(column, dataset[column].isnull().sum(), percentage))

missing(apr)

MISSING ROWS per COLUMN
marketplace: 0, 0.00%
customer_id: 0, 0.00%
review_id: 0, 0.00%
product_id: 0, 0.00%
product_parent: 0, 0.00%
product_title: 0, 0.00%
product_category: 0, 0.00%
star_rating: 0, 0.00%
helpful_votes: 0, 0.00%
downvotes: 0, 0.00%
total_votes: 0, 0.00%
vine: 0, 0.00%
verified_purchase: 0, 0.00%
review_headline: 59, 0.00%
review_body: 57, 0.00%
review_date: 257, 0.00%
purchased_counts: 0, 0.00%


In [ ]:
apr.dropna(axis=0, inplace=True)
missing(apr)

MISSING ROWS per COLUMN
marketplace: 0, 0.00%
customer_id: 0, 0.00%
review_id: 0, 0.00%
product_id: 0, 0.00%
product_parent: 0, 0.00%
product_title: 0, 0.00%
product_category: 0, 0.00%
star_rating: 0, 0.00%
helpful_votes: 0, 0.00%
downvotes: 0, 0.00%
total_votes: 0, 0.00%
vine: 0, 0.00%
verified_purchase: 0, 0.00%
review_headline: 0, 0.00%
review_body: 0, 0.00%
review_date: 0, 0.00%
purchased_counts: 0, 0.00%


In [ ]:
# Reset index and drop unimportant columns
apr = apr.reset_index().drop(['index', 'marketplace', 'vine'], axis=1)

## A Look into Our Final Dataset

In [ ]:
apr['verified_purchase'].value_counts()

Y    4365715
N    1468155
Name: verified_purchase, dtype: int64

In [ ]:
apr.groupby('star_rating')['star_rating'].count()

star_rating
1.0     347001
2.0     236052
3.0     454752
4.0    1074800
5.0    3721265
Name: star_rating, dtype: int64

In [ ]:
# An initial look at our dataframe
print('Reviews are taken from: {} to {}\n'. format(sorted(apr.review_date.head(1)).pop(),
                                                   sorted(apr.review_date.tail(1)).pop()))
print('Number of reviews: {:,}'.format(len(apr)))
print('Number of customers: {:,}'.format(len(apr.customer_id.unique())))
print('Number of product parents: {:,}'.format(len(apr.product_parent.unique())))
print('Number of product titles: {:,}'.format(len(apr.product_title.unique())))
print('Number of unique products: {:,}'.format(len(apr.product_id.unique())))
print('Number of product categories: {:,}'.format(len(apr.product_category.unique())))
print('Number of days a review was made on: {:,}'.format(len(apr.review_date.unique())))

Reviews are taken from: 1995-08-13 to 2015-08-31

Number of reviews: 5,833,870
Number of customers: 3,653,661
Number of product parents: 6,654
Number of product titles: 9,081
Number of unique products: 11,502
Number of product categories: 29
Number of days a review was made on: 6,838


In [ ]:
apr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5833870 entries, 0 to 5833869
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   customer_id        int64  
 1   review_id          object 
 2   product_id         object 
 3   product_parent     int64  
 4   product_title      object 
 5   product_category   object 
 6   star_rating        float64
 7   helpful_votes      float64
 8   downvotes          float64
 9   total_votes        float64
 10  verified_purchase  object 
 11  review_headline    object 
 12  review_body        object 
 13  review_date        object 
 14  purchased_counts   int64  
dtypes: float64(4), int64(3), object(8)
memory usage: 667.6+ MB


In [ ]:
apr.head()

,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,downvotes,total_votes,verified_purchase,review_headline,review_body,review_date,purchased_counts
0,53096384,R63J84G1LOX6R,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,4.0,0.0,1.0,1.0,N,ignore the review below,this is the first 8 issues of the series. it i...,1995-08-13,137
1,52741846,R3Q01UU0UZY3GH,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,5.0,0.0,0.0,0.0,N,Gaiman with yet another masterpiece,"This book, like most tpb's in the Vertigo line...",1998-04-15,137
2,52249743,R22W00B604VKCQ,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,5.0,0.0,0.0,0.0,N,"Breath taking, wonderful, amazing, mind tingling","Neil Gaiman, has taken a simple idea and turne...",1998-10-08,137
3,51796203,R1FKT8F96WD4JC,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,5.0,0.0,0.0,0.0,N,The beginning of a wonderful journey,"The Sandman is, in my opinion the best whole c...",1999-01-21,137
4,51864502,R1NDDVLJ1EWKYR,1563890119,763187671,The Sandman Vol. 1: Preludes and Nocturnes,Books,5.0,2.0,0.0,2.0,N,"Mythology, literature, and a damn good read!",I always hated comic books -- there was nothin...,1999-03-01,137


## Output Cleaned Dataset to New CSV File

In [ ]:
apr.to_csv('apr.csv')
from google.colab import files
files.download('apr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>